# Notebook 03: Comparison Engine (Layer 3)

This notebook demonstrates element-wise comparison between two PDF versions:
- **Text:** difflib structural diff + sentence-transformers semantic diff
- **Tables:** Cell-level diff with column alignment
- **Images:** SSIM + perceptual hashing + pixel overlay
- **Drawings:** Feature alignment + contour diff + dimension OCR

In [ ]:
import sys
sys.path.insert(0, '..')

import yaml
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from IPython.display import HTML, display

from src.comparators.text_comparator import TextComparator
from src.comparators.table_comparator import TableComparator
from src.comparators.image_comparator import ImageComparator
from src.comparators.drawing_comparator import DrawingComparator
from src.comparators.page_aligner import PageAligner

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

text_comp = TextComparator(config)
table_comp = TableComparator(config)
image_comp = ImageComparator(config)
drawing_comp = DrawingComparator(config)
print('All comparators loaded.')

## 3A. Text Comparison Demo

Compare text blocks with both structural and semantic diff.

In [ ]:
# Simulated text blocks from two document versions
old_texts = [
    ('p0_text_0_0', 'The maximum operating pressure is 150 PSI at ambient temperature.'),
    ('p0_text_0_100', 'Material specification: ASTM A105 forged carbon steel.'),
    ('p0_text_0_200', 'All welds shall be inspected per AWS D1.1 standards.'),
    ('p1_text_0_0', 'Installation requires a minimum clearance of 24 inches.'),
]

new_texts = [
    ('p0_text_0_0', 'The maximum operating pressure is 200 PSI at ambient temperature.'),  # Value changed
    ('p0_text_0_100', 'Material specification: ASTM A105 forged carbon steel.'),  # Unchanged
    ('p0_text_0_200', 'All welding work must comply with AWS D1.1 standards.'),  # Paraphrased
    # p1_text_0_0 removed
    ('p1_text_0_100', 'New section: Emergency shutdown procedures.'),  # Added
]

changes = text_comp.compare_text_blocks(old_texts, new_texts)

print(f'Found {len(changes)} text changes:\n')
for change in changes:
    print(f'  [{change.change_type.value.upper()}] {change.element_id}')
    if change.old_text:
        print(f'    Old: {change.old_text[:80]}...' if len(change.old_text) > 80 else f'    Old: {change.old_text}')
    if change.new_text:
        print(f'    New: {change.new_text[:80]}...' if len(change.new_text) > 80 else f'    New: {change.new_text}')
    if change.semantic_similarity > 0:
        print(f'    Semantic similarity: {change.semantic_similarity:.3f}')
        if change.is_semantic_only:
            print(f'    >> PARAPHRASE detected (meaning preserved)')
    print()

In [ ]:
# Display HTML diff for modified texts
for change in changes:
    if change.diff_html:
        print(f'\n--- {change.element_id} ---')
        display(HTML(change.diff_html))

## 3B. Table Comparison Demo

In [ ]:
# Simulated tables from two versions
old_table = pd.DataFrame({
    'Part Number': ['V-100', 'V-200', 'V-300', 'P-100'],
    'Description': ['Gate Valve', 'Ball Valve', 'Check Valve', 'Pump Assembly'],
    'Qty': [4, 2, 6, 1],
    'Material': ['CS', 'SS316', 'CS', 'CS'],
})

new_table = pd.DataFrame({
    'Part Number': ['V-100', 'V-200', 'V-300', 'P-100', 'V-400'],
    'Description': ['Gate Valve', 'Ball Valve', 'Check Valve', 'Pump Assembly', 'Butterfly Valve'],
    'Qty': [4, 3, 6, 1, 2],  # V-200 qty changed from 2 to 3
    'Material': ['CS', 'SS316', 'SS304', 'CS', 'SS316'],  # V-300 material changed
})

print('Old Table:')
print(old_table.to_string(index=False))
print('\nNew Table:')
print(new_table.to_string(index=False))

table_change = table_comp.compare_tables(old_table, new_table, 'p2_table_0_0')

print(f'\n=== Table Comparison Result ===')
print(f'Change type: {table_change.change_type.value}')
print(f'Added rows: {table_change.added_rows}')
print(f'Removed rows: {table_change.removed_rows}')
print(f'Modified cells: {table_change.modified_cells}')
print(f'Added columns: {table_change.added_columns}')
print(f'Removed columns: {table_change.removed_columns}')

## 3C. Image Comparison Demo

In [ ]:
# Create synthetic test images
def create_test_image(text='Test', color=(200, 200, 200), rect_pos=(100, 100, 300, 200)):
    img = np.ones((400, 500, 3), dtype=np.uint8) * 255
    cv2.rectangle(img, (rect_pos[0], rect_pos[1]), (rect_pos[2], rect_pos[3]), color, 2)
    cv2.putText(img, text, (150, 260), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    return img

old_img = create_test_image('Version 1', (0, 200, 0), (100, 100, 300, 200))
new_img = create_test_image('Version 2', (0, 0, 200), (120, 90, 320, 210))  # Slightly moved

img_change = image_comp.compare_images(
    old_img, new_img, element_id='p0_image_100_100', page_old=0, page_new=0
)

print(f'Image comparison result:')
print(f'  Change type: {img_change.change_type.value}')
print(f'  SSIM score: {img_change.ssim_score:.4f}')
print(f'  pHash distance: {img_change.phash_distance}')
print(f'  Ambiguous: {img_change.is_ambiguous}')
print(f'  Confidence: {img_change.confidence:.2f}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.imshow(cv2.cvtColor(old_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Old Image')
ax1.axis('off')
ax2.imshow(cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB))
ax2.set_title('New Image')
ax2.axis('off')
plt.suptitle(f'SSIM={img_change.ssim_score:.4f}, pHash dist={img_change.phash_distance}')
plt.tight_layout()
plt.show()

## 3D. Full Text Diff (Document Level)

In [ ]:
old_full = """Section 1: Introduction
This document describes the engineering specifications for Project Alpha.
The system operates at 150 PSI maximum pressure.

Section 2: Materials
All components shall be made from ASTM A105 carbon steel.
Gaskets: Spiral wound, 316SS inner ring."""

new_full = """Section 1: Introduction
This document describes the engineering specifications for Project Alpha.
The system operates at 200 PSI maximum pressure.

Section 2: Materials
All components shall be made from ASTM A105 carbon steel.
Gaskets: Spiral wound, 316SS inner ring.

Section 3: Safety
Emergency shutdown valves must be tested quarterly."""

full_diff = text_comp.compare_full_text(old_full, new_full)

print(f'Similarity ratio: {full_diff["similarity_ratio"]:.3f}')
print(f'Lines added: {full_diff["lines_added"]}')
print(f'Lines removed: {full_diff["lines_removed"]}')
print(f'\nUnified diff:')
print(full_diff['unified_diff'])

In [ ]:
print('Done! Comparison engine validated.')